# TV's Golden Age: Exploring Ratings and Popularity

## Introduction <a id='intro'></a>
This project explores data from the entertainment industry, focusing on television’s "Golden Age," which began in 1999 with the release of The Sopranos and continues today. By analyzing a dataset of TV shows, we aim to understand the relationship between audience engagement and critical reception. Specifically, we will examine how the number of votes a show receives correlates with its overall rating. The underlying assumption is that the most highly-rated shows from this era also attract the most audience participation, reflected in their vote counts. Movies will be excluded from this analysis to maintain a focused study on television trends.

In [38]:
# importing pandas
import pandas as pd


In [39]:
# reading the files and storing them to df
df = pd.read_csv('/datasets/movies_and_shows.csv')


Print the first 10 table rows:

In [40]:
df.head(10)


,name,Character,r0le,TITLE,Type,release Year,genres,imdb sc0re,imdb v0tes
0,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
1,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
2,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
3,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
4,Cybill Shepherd,Betsy,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
5,Peter Boyle,Wizard,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
6,Leonard Harris,Senator Charles Palantine,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
7,Diahnne Abbott,Concession Girl,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
8,Gino Ardito,Policeman at Rally,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0
9,Martin Scorsese,Passenger Watching Silhouette,ACTOR,Taxi Driver,MOVIE,1976,"['drama', 'crime']",8.2,808582.0


In [41]:
# obtaining general information about the data in df
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85579 entries, 0 to 85578
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0      name       85579 non-null  object 
 1   Character     85579 non-null  object 
 2   r0le          85579 non-null  object 
 3   TITLE         85578 non-null  object 
 4     Type        85579 non-null  object 
 5   release Year  85579 non-null  int64  
 6   genres        85579 non-null  object 
 7   imdb sc0re    80970 non-null  float64
 8   imdb v0tes    80853 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 5.9+ MB


The table contains nine columns. The majority store the same data type: object. The only exceptions are `'release Year'` (int64 type), `'imdb sc0re'` (float64 type) and `'imdb v0tes'` (float64 type). Scores and votes will be used in our analysis, so it's important to verify that they are present in the dataframe in the appropriate numeric format. Three columns (`'TITLE'`, `'imdb sc0re'` and `'imdb v0tes'`) have missing values.

According to the documentation:
- `'name'` — actor/director's name and last name
- `'Character'` — character played (for actors)
- `'r0le '` — the person's contribution to the title (it can be in the capacity of either actor or director)
- `'TITLE '` — title of the movie (show)
- `'  Type'` — show or movie
- `'release Year'` — year when movie (show) was released
- `'genres'` — list of genres under which the movie (show) falls
- `'imdb sc0re'` — score on IMDb
- `'imdb v0tes'` — votes on IMDb

We can see three issues with the column names:
1. Some names are uppercase, while others are lowercase.
2. There are names containing whitespace.
3. A few column names have digit '0' instead of letter 'o'. 


### Conclusions <a id='data_review_conclusions'></a> 

Each row in the table stores data about a movie or show. The columns can be divided into two categories: the first is about the roles held by different people who worked on the movie or show (role, name of the actor or director, and character if the row is about an actor); the second category is information about the movie or show itself (title, release year, genre, imdb figures).

It's clear that there is sufficient data to do the analysis and evaluate our assumption. However, to move forward, we need to preprocess the data.

## Data preprocessing <a id='data_preprocessing'></a>


In [42]:
# the list of column names in the df table
df.columns


Index(['   name', 'Character', 'r0le', 'TITLE', '  Type', 'release Year',
       'genres', 'imdb sc0re', 'imdb v0tes'],
      dtype='object')

In [43]:
# renaming columns
df= df.rename(columns={
    '   name':'name',
    'Character':'character',
    'r0le':'role',
    'TITLE':'title',
    '  Type':'type',
    'release Year':'release_year', 
    'genres':'genres',
    'imdb sc0re':'imdb_score',
    'imdb v0tes':'imdb_votes'
})


In [44]:
# checking result: the list of column names
df.columns


Index(['name', 'character', 'role', 'title', 'type', 'release_year', 'genres',
       'imdb_score', 'imdb_votes'],
      dtype='object')

### Missing values <a id='missing_values'></a>

In [45]:
# calculating missing values

df.isna().sum()

name               0
character          0
role               0
title              1
type               0
release_year       0
genres             0
imdb_score      4609
imdb_votes      4726
dtype: int64

<div class="alert alert-success">
<b>Reviewer's comment v1</b>
    
Great that you selected `isna()` method to find missing values! 

It is also sometimes helpful to check not only the total amount of missing values in each column but also look at the percentage of missing values. It helps to understand the overall impact. You can check percentage using, for example, this code:

`df.isnull().sum()/len(df)`

Not all missing values affect the research: the single missing value in `'title'` is not critical. The missing values in columns `'imdb_score'` and `'imdb_votes'` represent around 6% of all records (4,609 and 4,726, respectively, of the total 85,579). This could potentially affect our research. To avoid this issue, we will drop rows with missing values in the `'imdb_score'` and `'imdb_votes'` columns.

In [46]:
# dropping rows where columns with title, scores and votes have missing values
df = df.dropna(axis='rows')

In [47]:
# counting missing values
df.isna().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80853 entries, 0 to 85578
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          80853 non-null  object 
 1   character     80853 non-null  object 
 2   role          80853 non-null  object 
 3   title         80853 non-null  object 
 4   type          80853 non-null  object 
 5   release_year  80853 non-null  int64  
 6   genres        80853 non-null  object 
 7   imdb_score    80853 non-null  float64
 8   imdb_votes    80853 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 6.2+ MB


### Duplicates <a id='duplicates'></a>

In [48]:
# counting duplicate rows
duplicated_rows = df.duplicated().sum()
duplicated_rows

6994

Review the duplicate rows to determine if removing them would distort our dataset.

In [49]:
duplicated_rows = df[df.duplicated(keep=False)]
duplicated_rows.head().reset_index()

,index,name,character,role,title,type,release_year,genres,imdb_score,imdb_votes
0,7560,Philip Greene,Baseball Fan (uncredited),ACTOR,How Do You Know,MOVIE,2010,"['comedy', 'drama', 'romance']",5.4,50383.0
1,7561,Philip Greene,Baseball Fan (uncredited),ACTOR,How Do You Know,MOVIE,2010,"['comedy', 'drama', 'romance']",5.4,50383.0
2,14502,Dan Levy,Reporter,ACTOR,A Very Harold & Kumar Christmas,MOVIE,2011,"['comedy', 'fantasy', 'romance']",6.2,69562.0
3,14512,Dan Levy,Reporter,ACTOR,A Very Harold & Kumar Christmas,MOVIE,2011,"['comedy', 'fantasy', 'romance']",6.2,69562.0
4,18951,Nicolas Le Nev??,unknown,DIRECTOR,Sammy & Co,SHOW,2014,"['animation', 'european']",5.7,31.0


There are clear duplicates in the printed rows. We can safely remove them.

In [50]:
# removing duplicate rows
duplicated_rows = duplicated_rows.drop_duplicates()


Check for duplicate rows once more to make sure you have removed all of them:

In [51]:
# checking for duplicates
duplicated_rows.head()


,name,character,role,title,type,release_year,genres,imdb_score,imdb_votes
7560,Philip Greene,Baseball Fan (uncredited),ACTOR,How Do You Know,MOVIE,2010,"['comedy', 'drama', 'romance']",5.4,50383.0
14502,Dan Levy,Reporter,ACTOR,A Very Harold & Kumar Christmas,MOVIE,2011,"['comedy', 'fantasy', 'romance']",6.2,69562.0
18951,Nicolas Le Nev??,unknown,DIRECTOR,Sammy & Co,SHOW,2014,"['animation', 'european']",5.7,31.0
22454,John Iii Franklin,Himself,ACTOR,Last Chance U,SHOW,2016,"['documentation', 'sport']",8.4,6897.0
29556,Claudio Roca,Nicol?­s,ACTOR,Narcos: Mexico,SHOW,2018,"['drama', 'crime']",8.4,82042.0


In [52]:
# viewing unique type names
unique_type = df['type'].unique()
unique_type

array(['MOVIE', 'the movie', 'tv show', 'shows', 'movies', 'tv shows',
       'tv series', 'tv', 'SHOW'], dtype=object)

I will look through the list to find implicit duplicates of `'show'` (`'movie'` duplicates will be ignored since the assumption is about shows). These could be names written incorrectly or alternative names of the same genre.

You will see the following implicit duplicates:
* `'shows'`
* `'SHOW'`
* `'tv show'`
* `'tv shows'`
* `'tv series'`
* `'tv'`

To get rid of them, declare the function `replace_wrong_show()` with two parameters: 
* `wrong_shows_list=` — the list of duplicates
* `correct_show=` — the string with the correct value

The function should correct the names in the `'type'` column from the `df` table (i.e., replace each value from the `wrong_shows_list` list with the value in `correct_show`).

In [53]:
# function for replacing implicit duplicates
wrong_shows_list = ['shows', 'tv show', 'tv shows', 'tv series', 'tv']
correct_show = 'SHOW'

def replace_wrong_show(wrong_shows_list, correct_show):
    for name in wrong_shows_list:
        df['type'] = df['type'].replace(name, correct_show)
    

I called `replace_wrong_show()` and passed it arguments so that it clears implicit duplicates and replaces them with `SHOW`:

In [54]:
# removing implicit duplicates
replace_wrong_show(wrong_shows_list, correct_show)

Make sure the duplicate names are removed. Print the list of unique values from the `'type'` column:

In [55]:
# viewing unique genre names
df['type'].sort_values().unique()

array(['MOVIE', 'SHOW', 'movies', 'the movie'], dtype=object)

### Conclusions <a id='data_preprocessing_conclusions'></a>
We detected three issues with the data:

- Incorrect header styles
- Missing values
- Duplicate rows and implicit duplicates

The headers have been cleaned up to make processing the table simpler.

All rows with missing values have been removed. 

The absence of duplicates will make the results more precise and easier to understand.

Now we can move on to our analysis of the prepared data.

## Stage 3. Data analysis <a id='hypotheses'></a>

Based on the previous project stages, you can now define how the assumption will be checked. Calculate the average amount of votes for each score (this data is available in the `imdb_score` and `imdb_votes` columns), and then check how these averages relate to each other. If the averages for shows with the highest scores are bigger than those for shows with lower scores, the assumption appears to be true.

Based on this, complete the following steps:

- Filter the dataframe to only include shows released in 1999 or later.
- Group scores into buckets by rounding the values of the appropriate column (a set of 1-10 integers will help us make the outcome of our calculations more evident without damaging the quality of our research).
- Identify outliers among scores based on their number of votes, and exclude scores with few votes.
- Calculate the average votes for each score and check whether the assumption matches the results.

To filter the dataframe and only include shows released in 1999 or later, you will take two steps. First, keep only titles published in 1999 or later in our dataframe. Then, filter the table to only contain shows (movies will be removed).

In [56]:
# using conditional indexing modify df so it has only titles released after 1999 (with 1999 included)
# give the slice of dataframe new name

show_year = df[df['release_year'] >= 1999].sort_values(by='release_year', ascending=True)
show_year



,name,character,role,title,type,release_year,genres,imdb_score,imdb_votes
4060,Manisha Koirala,Priya Verma,ACTOR,Mann,MOVIE,1999,"['drama', 'romance']",6.1,5451.0
4142,Sooraj Barjatya,unknown,DIRECTOR,Hum Saath Saath Hain,MOVIE,1999,"['romance', 'family', 'fantasy', 'drama']",6.2,8552.0
4141,Zoya Afroz,Radhika,ACTOR,Hum Saath Saath Hain,MOVIE,1999,"['romance', 'family', 'fantasy', 'drama']",6.2,8552.0
4140,Jatin Kanakia,Dr. Sen,ACTOR,Hum Saath Saath Hain,MOVIE,1999,"['romance', 'family', 'fantasy', 'drama']",6.2,8552.0
4139,Achyut Potdar,Khan Saheb,ACTOR,Hum Saath Saath Hain,MOVIE,1999,"['romance', 'family', 'fantasy', 'drama']",6.2,8552.0
...,...,...,...,...,...,...,...,...,...
67911,Jean Harlow,Self (archive footage),ACTOR,The Mystery of Marilyn Monroe: The Unheard Tapes,MOVIE,2022,['documentation'],6.2,3978.0
67912,Jack Lemmon,Self (archive footage),ACTOR,The Mystery of Marilyn Monroe: The Unheard Tapes,MOVIE,2022,['documentation'],6.2,3978.0
67913,Jane Russell,Self (archive footage),ACTOR,The Mystery of Marilyn Monroe: The Unheard Tapes,MOVIE,2022,['documentation'],6.2,3978.0
67904,Lauren Bacall,Self (archive footage),ACTOR,The Mystery of Marilyn Monroe: The Unheard Tapes,MOVIE,2022,['documentation'],6.2,3978.0


In [57]:
# repeat conditional indexing so df has only shows (movies are removed as result)
show_type_year = show_year[show_year['type'] == 'SHOW']
show_type_year


,name,character,role,title,type,release_year,genres,imdb_score,imdb_votes
2076,Mayumi Tanaka,Monkey D. Luffy (voice),ACTOR,One Piece,SHOW,1999,"['animation', 'action', 'comedy', 'drama', 'fa...",8.8,117129.0
2077,Kazuya Nakai,Roronoa Zoro (voice),ACTOR,One Piece,SHOW,1999,"['animation', 'action', 'comedy', 'drama', 'fa...",8.8,117129.0
2078,Akemi Okamura,Nami (voice),ACTOR,One Piece,SHOW,1999,"['animation', 'action', 'comedy', 'drama', 'fa...",8.8,117129.0
2079,Kappei Yamaguchi,Usopp (voice),ACTOR,One Piece,SHOW,1999,"['animation', 'action', 'comedy', 'drama', 'fa...",8.8,117129.0
2080,Hiroaki Hirata,Vinsmoke Sanji (voice),ACTOR,One Piece,SHOW,1999,"['animation', 'action', 'comedy', 'drama', 'fa...",8.8,117129.0
...,...,...,...,...,...,...,...,...,...
71830,Gianluca Gobbi,Luca,ACTOR,"Devotion, a Story of Love and Desire",SHOW,2022,"['drama', 'romance']",6.4,1500.0
71826,Alice Arcuri,Gioia,ACTOR,"Devotion, a Story of Love and Desire",SHOW,2022,"['drama', 'romance']",6.4,1500.0
71825,Leonardo Carradori,Lorenzo,ACTOR,"Devotion, a Story of Love and Desire",SHOW,2022,"['drama', 'romance']",6.4,1500.0
71823,Michele Riondino,Carlo Pentecoste,ACTOR,"Devotion, a Story of Love and Desire",SHOW,2022,"['drama', 'romance']",6.4,1500.0


The scores that are to be grouped should be rounded. For instance, titles with scores like 7.8, 8.1, and 8.3 will all be placed in the same bucket with a score of 8.

In [58]:
# rounding column with scores
show_type_year = show_type_year.copy()
show_type_year.loc[:,'rounded_score'] = show_type_year['imdb_score'].round()
show_type_year[['imdb_score', 'rounded_score']].head()

,imdb_score,rounded_score
2076,8.8,9.0
2077,8.8,9.0
2078,8.8,9.0
2079,8.8,9.0
2080,8.8,9.0


It is now time to identify outliers based on the number of votes.

In [59]:
# Use groupby() for scores and count all unique values in each group, print the result
grouped_scores = show_type_year.groupby('rounded_score')['imdb_score'].count()
grouped_scores

rounded_score
2.0       40
3.0       27
4.0      225
5.0      691
6.0     2986
7.0     5322
8.0     5200
9.0      583
10.0       8
Name: imdb_score, dtype: int64

Based on the aggregation performed, it is evident that scores 2 (24 voted shows), 3 (27 voted shows), and 10 (only 8 voted shows) are outliers. There isn't enough data for these scores for the average number of votes to be meaningful.

To obtain the mean numbers of votes for the selected scores (we identified a range of 4-9 as acceptable), use conditional filtering and grouping.

In [60]:
# filter dataframe using two conditions (scores to be in the range 4-9)
filtered_df = show_type_year[(show_type_year['rounded_score'] >= 4) & (show_type_year['rounded_score'] <= 9)]
filtered_df.sort_values(by='rounded_score', ascending=True)


# group scores and corresponding average number of votes, reset index and print the result
grouped_votes = filtered_df.groupby('rounded_score')['imdb_votes'].mean().reset_index()
grouped_votes




,rounded_score,imdb_votes
0,4.0,6544.480000
1,5.0,2846.192475
2,6.0,3168.731413
3,7.0,8012.183953
4,8.0,28361.942115
5,9.0,121401.017153


Now for the final step! Round the column with the averages, rename both columns, and print the dataframe in descending order.

In [61]:
# round column with averages
grouped_votes['imdb_votes'] = grouped_votes['imdb_votes'].round()
grouped_votes

# rename columns
grouped_votes = grouped_votes.rename(columns={'rounded_score': 'score', 'imdb_votes': 'average_votes'})
grouped_votes

# print dataframe in descending order
grouped_votes = grouped_votes.sort_values(by='average_votes', ascending=False)
grouped_votes


,score,average_votes
5,9.0,121401.0
4,8.0,28362.0
3,7.0,8012.0
0,4.0,6544.0
2,6.0,3169.0
1,5.0,2846.0


The assumption macthes the analysis: the shows with the top 3 scores have the most amounts of votes.

## Conclusion <a id='hypotheses'></a>

The research done confirms that highly-rated shows released during the "Golden Age" of television also have the most votes. While shows with score 4 have more votes than ones with scores 5 and 6, the top three (scores 7-9) have the largest number. The data studied represents around 94% of the original set, so we can be confident in our findings.